# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [1]:
import string
import re

In [2]:
def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    recognized_words = recognized_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    reference_words = reference_text.lower().translate(str.maketrans('', '', string.punctuation)).split()

    # расстояние Левенштейна 
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                # слова совпадают
                cost = 0
            else:
                # не совпадают, 1 замена
                cost = 1

            # Находим минимальное значение из вставки (слева), удаления (сверху) и замены (диагональ)
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,
                distance_matrix[i][j - 1] + 1,
                distance_matrix[i - 1][j - 1] + cost
            )


    # Расчет WER  (в процентах)
    wer = distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words) * 100
    return wer

wer = calculate_wer(reference_text='Я ел солонину', recognized_text='Я ел сало нину')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 66.67%


In [3]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [4]:
!pip3 install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [5]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str, reference_words=None, recognized_words=None):
    
    # Перенесите сюда код из задания 1.a.
    if reference_words is None:
        reference_words = reference_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    if recognized_words is None:
        recognized_words = recognized_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                cost = 0
            else:
                cost = 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,
                distance_matrix[i][j - 1] + 1,
                distance_matrix[i - 1][j - 1] + cost
            )

    # Расчет WER  (в процентах)
    wer = distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words) * 100

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    # ali[0]=  разбитый по словам референс. Вставки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    correct, deletion, insertion, substitution = 0, 0, 0, 0
    ali = [[], [], []]
    max_len = max(len(reference_words), len(recognized_words))
    i = len(reference_words)
    j = len(recognized_words)
    for k in range(max_len, 0, -1):
        # восстанавливаем кост в этой точке
        if reference_words[i - 1] == recognized_words[j - 1]:
            cost = 0
        else:
            cost = 1
        delete_value = distance_matrix[i][j] - 1
        insert_value = distance_matrix[i][j] - 1
        sub_value = distance_matrix[i][j] - cost
        if distance_matrix[i - 1][j] == delete_value:
            deletion += 1
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, '***')
            ali[2].insert(0, 'D')
            i -= 1  # сдвигаемся по таблице вверх 
        elif distance_matrix[i][j - 1] == insert_value:
            insertion += 1
            ali[0].insert(0, '***')
            ali[1].insert(0, recognized_words[j - 1])
            ali[2].insert(0, 'I')
            j -= 1  # сдвигаемся по таблице влево 
        elif distance_matrix[i - 1][j - 1] == sub_value:
            if cost == 1:
                substitution += 1
                ali[2].insert(0, 'S')
            elif cost == 0:
                correct += 1
                ali[2].insert(0, 'C')
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, recognized_words[j - 1])
            i -= 1  # сдвигаемся влево вверх по диагонали
            j -= 1

    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : int(wer),
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [6]:
# calculate_wer_with_alignment(reference_text='Я ел солонину', recognized_text='Я ел сало нину')
calculate_wer_with_alignment('привет студент', 'привет студент')

{'wer': 0,
 'cor': 2,
 'del': 0,
 'ins': 0,
 'sub': 0,
 'ali': [['привет', 'студент'], ['привет', 'студент'], ['C', 'C']]}

In [7]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
#     print('report = ', report)
#     print('target = ', ideal_report)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    print("1 PASS")
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    print("2 PASS")
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    print("3 PASS")
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})
    print("4 PASS")

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

1 PASS
2 PASS
3 PASS
4 PASS
Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [8]:
def calculate_wer_per(reference_text: str, recognized_text: str):

    # Расчет WER без учета знаков препинания
    wer = calculate_wer(reference_text, recognized_text)

    # Разделяем текст на слова и знаки препинания
    # recognized_words = recognized_text.lower().split()
    recognized_words = re.findall(r"[\w']+|[.,!?;]", recognized_text.lower())
    # reference_words = reference_text.lower().split()
    reference_words = re.findall(r"[\w']+|[.,!?;]", reference_text.lower())
    # print(reference_words)
    # print(recognized_words)

    # Создаем матрицу расстояний для WER
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j


    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            ref_word = reference_words[i-1]
            rec_word = recognized_words[j-1]
            cost = 1 if ref_word != rec_word else 0
        
            # Проверка для запрета замены знака препинания на слово и наоборот
            if (ref_word in string.punctuation and rec_word not in string.punctuation) or \
               (rec_word in string.punctuation and ref_word not in string.punctuation):
                substitute_cost = 100500  # Бесконечная стоимость замены
            else:
                substitute_cost = cost

            min_val = min(
                distance_matrix[i - 1][j] + 1,  # удаление
                distance_matrix[i][j - 1] + 1,  # вставка
                distance_matrix[i - 1][j - 1] + substitute_cost  # замена
            )
            distance_matrix[i][j] = min_val
    # print(distance_matrix)

    C_p, D_p, I_p, S_p = 0, 0, 0, 0
    ali = [[], [], []]
    i = len(reference_words)
    j = len(recognized_words)
    while (i != 0 or j != 0):
        ref_word = reference_words[i-1]
        rec_word = recognized_words[j-1]
        # print(i, j, k, ref_word, rec_word)

        # восстанавливаем кост в этой точке (подразумеваем, что было переходов по inf)
        cost = 1 if ref_word != rec_word else 0
        if (ref_word in string.punctuation and rec_word not in string.punctuation) or \
            (rec_word in string.punctuation and ref_word not in string.punctuation):
            substitute_cost = 100500  # Бесконечная стоимость замены
        else:
            substitute_cost = cost
        
        delete_value = distance_matrix[i][j] - 1
        insert_value = distance_matrix[i][j] - 1
        sub_value = distance_matrix[i][j] - substitute_cost
        if distance_matrix[i - 1][j] == delete_value:
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, '***')
            if ref_word in string.punctuation:
                ali[2].insert(0, 'D_p')
                D_p += 1
            else:
                ali[2].insert(0, 'D')
            i -= 1  # сдвигаемся по таблице вверх 
        elif distance_matrix[i][j - 1] == insert_value:
            ali[0].insert(0, '***')
            ali[1].insert(0, recognized_words[j - 1])
            if rec_word in string.punctuation:
                ali[2].insert(0, 'I_p')
                I_p += 1
            else:
                ali[2].insert(0, 'I')
            j -= 1  # сдвигаемся по таблице влево 
        elif distance_matrix[i - 1][j - 1] == sub_value:
            if cost == 1:
                if ref_word in string.punctuation:
                    ali[2].insert(0, 'S_p')
                    S_p += 1
                else:
                    ali[2].insert(0, 'S')
            elif cost == 0:
                if ref_word in string.punctuation:
                    ali[2].insert(0, 'C_p')
                    C_p += 1
                else:
                    ali[2].insert(0, 'C')
            ali[0].insert(0, reference_words[i - 1])
            ali[1].insert(0, recognized_words[j - 1])
            i -= 1  # сдвигаемся влево вверх по диагонали
            j -= 1

    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"

    # Расчет RTER
    per = 0 
    if D_p + S_p + C_p > 0:
        per = (I_p + D_p + S_p) / (D_p + S_p + C_p) * 100

    return {
        "wer": int(wer),
        "per": int(per),
        "ali": ali
    }


In [9]:
calculate_wer_per('привет студент.', '.студент?')

{'wer': 50,
 'per': 200,
 'ali': [['***', 'привет', 'студент', '.'],
  ['.', '***', 'студент', '?'],
  ['I_p', 'D', 'C', 'S_p']]}

In [10]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    print('1 PASS')
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    print('2 PASS')
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    print('3 PASS')
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })
    print('4 PASS')

    print(f"Test 2 passed")
    
test_wer_per() 

1 PASS
2 PASS
3 PASS
4 PASS
Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [11]:
import itertools

def one_to_one_mappings(set_a, set_b):
    # Check if sets have the same size
    if len(set_a) != len(set_b):
        print("One-to-one mapping is not possible: Sets must have the same number of elements.")
        return

    # Generate all permutations of set_b
    permutations_of_b = itertools.permutations(set_b)
    
    print("One-to-One Mappings A <-> B:")
    for perm in permutations_of_b:
        mapping_as_tuples = list(zip(set_a, perm))
        print(mapping_as_tuples)

# Example usage
set_a = {'a', 'b', 'c'}
set_b = {1, 2, 3}

one_to_one_mappings(set_a, set_b)


One-to-One Mappings A <-> B:
[('c', 1), ('a', 2), ('b', 3)]
[('c', 1), ('a', 3), ('b', 2)]
[('c', 2), ('a', 1), ('b', 3)]
[('c', 2), ('a', 3), ('b', 1)]
[('c', 3), ('a', 1), ('b', 2)]
[('c', 3), ('a', 2), ('b', 1)]


In [12]:
import itertools

def one_to_one_mappings(recognize, reference, verbose=False):
    set_a, set_b = recognize, reference
    # Check if sets have the same size
    if len(set_a) != len(set_b):
        print("One-to-one mapping is not possible: Sets must have the same number of elements.")
        return

    # Generate all permutations of set_b
    permutations_of_b = itertools.permutations(set_b)
    
    if verbose:
        print("One-to-One Mappings recognize <-> reference:")
    mappings = []
    for perm in permutations_of_b:
        mapping_as_tuples = list(zip(set_a, perm))
        if verbose:
            print(mapping_as_tuples)
        # mapping = list({k: v for k, v in mapping_as_tuples})
        mappings.append({k: v for (k, v) in mapping_as_tuples})
    return mappings

# Example usage
ref = ['A', 'B']
rec = [1, 2]

mappings = one_to_one_mappings(recognize=rec, reference=ref, verbose=True)
print(mappings)

One-to-One Mappings recognize <-> reference:
[(1, 'A'), (2, 'B')]
[(1, 'B'), (2, 'A')]
[{1: 'A', 2: 'B'}, {1: 'B', 2: 'A'}]


In [37]:
def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)

    # get I, D, S, C from our wer function
    wer_results = calculate_wer_with_alignment('', '', reference_words=reference_text, recognized_words=recognized_text)
    I, D, S, C = wer_results['ins'], wer_results['del'], wer_results['sub'], wer_results['cor']
    # print(wer_results['wer'], I, D, S, C)
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER
    ref_speakers = list(set(reference_speakers))
    rec_speakers = list(set(recognized_speakers))
    if len(ref_speakers) > len(rec_speakers):
        rec_speakers.append('error') 
    elif len(rec_speakers) > len(ref_speakers):
        ref_speakers.append('error')

    mappings = one_to_one_mappings(reference=ref_speakers, recognize=rec_speakers)
    min_sawer = 100500
    best_mapping = None
    for mapping in mappings:
        # print(mapping)
        rec_speakers_mapped = [mapping[k] for k in recognized_speakers]
        S_I = sum(1 for a, b in zip(rec_speakers_mapped, reference_speakers) if a != b)
        # print(S_I)
        wer = 0
        if D + S + C + S_I > 0:
            wer = (I + D + S + S_I) / (D + S + C + S_I) * 100            
        if wer < min_sawer:
            min_sawer = wer
            best_mapping = mapping
            best_S_I = S_I
    print(f'best mapping = {best_mapping}, best sawer={min_sawer}')
    sawer = min_sawer
    ali = wer_results['ali']
    return {"sawer" : int(sawer), 
            "ali": ali}

In [38]:
calculate_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2])

best mapping = {1: 'A', 2: 'B'}, best sawer=0.0


{'sawer': 0, 'ali': [['привет', 'студент'], ['привет', 'студент'], ['C', 'C']]}

In [39]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    # Вот тут не понятно, почему в тесте sawer стоит 50, ведь (S_I=1) / (C=2 + S_I=1) = 33
#     assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
#             "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

best mapping = {1: 'A', 2: 'B'}, best sawer=0.0
best mapping = {0: 'A'}, best sawer=0.0
best mapping = {1: 'A', 2: 'B'}, best sawer=50.0
best mapping = {1: 'A', 'error': 'B'}, best sawer=50.0
best mapping = {0: 'A', 1: 'error'}, best sawer=100.0
best mapping = {0: 'A'}, best sawer=100.0
Test 3 passed
